**Simple example for Asg1 with handcrafted feature**

An example diagram for instance search.
Please read the slide "Information for Asg1" first.

In every bounding box file (the txt files), each line records the coordinates of one bounding box in format: x of top-left point, y of top-left point, width, height.

The bounding box information is only provided for the query images.

Notes: it's possible that there are more than one instances in a query image (see the annotation(s) in the corresponding txt file), you should find the most similar image rank list in the gallery by considering all the instances.

In [1]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 

Input 

Initialize the necessary parameters, including paths, feature extractors.

In [2]:
path_query=r'/Users/srwang/Downloads/asg1/query'
path_query_txt=r'/Users/srwang/Downloads/asg1/query_txt'
# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=r'/Users/srwang/Downloads/asg1/gallery'
name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)
sift = cv2.ORB_create()
record_all=np.zeros((num_query,len(name_gallery)))

Process

Feature extraction -> calculate distance -> compare

In [3]:
# the iteration loop for query 
# iteration is 2 here, only for the demonstration
for i in range(2):
    time_s = time.time()
    dist_record=[]
    per_query_name=path_query+'/'+str(i)+'.jpg'
    per_query=cv2.imread(per_query_name)
    # feature extraction for per query
    # the bounding box information is not considered
    # quite naive, just an example
    per_query_kp, per_query_des = sift.detectAndCompute(per_query,None)
    # the iteration loop for gallery
    for j in range(num_gallery):
        per_gallery_name = path_gallery+'/'+str(j)+'.jpg'
        per_gallery=cv2.imread(per_gallery_name)
        # feature extraction for per gallery
        per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
        # use part of the features to make the calculation feasible
        # quite naive, just an example
        min_kp_num=np.amin([len(per_query_kp),len(per_gallery_kp)])
        query_part=per_query_des[0:min_kp_num,:]
        gallery_part=per_gallery_des[0:min_kp_num,:]
        # distance calculation in feature domain (similarity)
        dist_record.append(np.sum((np.double(query_part)-np.double(gallery_part))**2)/np.prod(np.shape(query_part)))
    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
    # update the results for one query 
    record_all[i,:]=ascend_index
    time_e = time.time()
    print('retrieval time for query {} is {}s'.format(i, time_e-time_s))

retrival time for query 0 is 472.3902609348297s
retrival time for query 1 is 449.0376470088959s


Output 

Save the rank list file.

In [4]:
# write the output file following the example
f=open(r'./rank_list.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()

In [5]:
record_all

array([[21181., 12091.,  2073., ..., 21919., 21002.,  4307.],
       [  802.,  7070., 14675., ...,  3103., 15930.,  7421.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])